In [1]:
!pkill -f ollama
!sleep 2


In [2]:
!apt-get update && apt-get install -y zstd

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,341 kB]
Get:6 https://cli.github.com/packages stable/main amd64 Packages [356 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:9 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [85.0 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,712 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,677 kB]
Get:13 https://ppa.launchpadcontent.net/deadsnakes/p

In [3]:
!curl -fsSL https://ollama.com/install.sh | sh


>>> Installing ollama to /usr/local
>>> Downloading ollama-linux-amd64.tar.zst
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [4]:
ollama_model_id = "qwen2.5:7b-instruct-q4_0"

In [5]:
!nohup bash -c "OLLAMA_HOST=0.0.0.0:8000 OLLAMA_ORIGIN=* ollama serve" &
!sleep 5 && tail /content/nohup.out

nohup: appending output to 'nohup.out'
time=2026-02-08T13:21:35.896Z level=INFO source=images.go:473 msg="total blobs: 0"
time=2026-02-08T13:21:35.896Z level=INFO source=images.go:480 msg="total unused blobs removed: 0"
time=2026-02-08T13:21:35.896Z level=INFO source=routes.go:1689 msg="Listening on [::]:8000 (version 0.15.6)"
time=2026-02-08T13:21:35.909Z level=INFO source=runner.go:67 msg="discovering available GPUs..."
time=2026-02-08T13:21:35.910Z level=INFO source=runner.go:106 msg="experimental Vulkan support disabled.  To enable, set OLLAMA_VULKAN=1"
time=2026-02-08T13:21:35.911Z level=INFO source=server.go:431 msg="starting runner" cmd="/usr/local/bin/ollama runner --ollama-engine --port 41959"
time=2026-02-08T13:21:36.710Z level=INFO source=server.go:431 msg="starting runner" cmd="/usr/local/bin/ollama runner --ollama-engine --port 43305"
time=2026-02-08T13:21:36.842Z level=INFO source=server.go:431 msg="starting runner" cmd="/usr/local/bin/ollama runner --ollama-engine --port

In [6]:
#!ollama pull {ollama_model_id}

!OLLAMA_HOST=localhost:8000 ollama pull qwen2.5:7b-instruct-q4_0


In [7]:
!OLLAMA_HOST=localhost:8000 ollama list


NAME                        ID              SIZE      MODIFIED               
qwen2.5:7b-instruct-q4_0    2e92ac0dd3a8    4.4 GB    Less than a second ago    


In [8]:
!nohup ollama serve &


nohup: appending output to 'nohup.out'


In [9]:
!ollama pull qwen2.5:7b-instruct-q4_0


In [10]:
!nohup bash -c "OLLAMA_HOST=0.0.0.0:8000 OLLAMA_ORIGIN=* ollama run {ollama_model_id}"  &
!sleep 5 && tail /content/nohup.out

nohup: appending output to 'nohup.out'
⠹ ⠸ ⠼ ⠦ ⠦ ⠇ llama_context:      CUDA0 compute buffer size =   304.00 MiB
llama_context:  CUDA_Host compute buffer size =    15.01 MiB
llama_context: graph nodes  = 959
llama_context: graph splits = 2
time=2026-02-08T13:22:34.054Z level=INFO source=server.go:1388 msg="llama runner started in 4.14 seconds"
time=2026-02-08T13:22:34.054Z level=INFO source=sched.go:537 msg="loaded runners" count=1
time=2026-02-08T13:22:34.054Z level=INFO source=server.go:1350 msg="waiting for llama runner to start responding"
time=2026-02-08T13:22:34.054Z level=INFO source=server.go:1388 msg="llama runner started in 4.14 seconds"
[GIN] 2026/02/08 - 13:22:34 | 200 |  4.847674677s |       127.0.0.1 | POST     "/api/generate"


In [11]:
%%bash
curl http://localhost:8000/api/chat \
  -H "Content-Type: application/json" \
  -d '{
    "model": "qwen2.5:7b-instruct-q4_0",
    "stream": false,
    "messages": [
      { "role": "user", "content": "ما هي عاصمة مصر؟" }
    ]
  }'


{"model":"qwen2.5:7b-instruct-q4_0","created_at":"2026-02-08T13:22:34.682872463Z","message":{"role":"assistant","content":"عاصمة مصر هي القاهرة."},"done":true,"done_reason":"stop","total_duration":507205625,"load_duration":155925599,"prompt_eval_count":36,"prompt_eval_duration":119390916,"eval_count":8,"eval_duration":199441518}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   507  100   347  100   160    683    314 --:--:-- --:--:-- --:--:--  1000


In [12]:
!pip install pyngrok

In [13]:
from google.colab import userdata
from pyngrok import ngrok, conf

# Retrieve the ngrok auth token from user data
ngrok_auth_token = userdata.get('colab-ngrok')

# Assign the auth token to the pyngrok configuration
conf.get_default().auth_token = ngrok_auth_token

port = "8000"

# Connect to ngrok and print the public URL
public_url = ngrok.connect(port).public_url
print(public_url)

https://ffae-34-125-192-136.ngrok-free.app


In [15]:
%%bash
curl -X POST https://ffae-34-125-192-136.ngrok-free.app/api/chat \
  -H "Content-Type: application/json" \
  -d '{
    "model": "qwen2.5:7b-instruct-q4_0",
    "stream": false,
    "messages": [
      { "role": "user", "content": "ما عاصمة قطر ؟" }
    ]
  }'

{"model":"qwen2.5:7b-instruct-q4_0","created_at":"2026-02-08T13:23:05.593915953Z","message":{"role":"assistant","content":"عاصمة قطر هي الدوحة."},"done":true,"done_reason":"stop","total_duration":536879880,"load_duration":152188877,"prompt_eval_count":36,"prompt_eval_duration":104623292,"eval_count":10,"eval_duration":245483168}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   502  100   346  100   156    429    193 --:--:-- --:--:-- --:--:--   622
